# Gun Violence in the U.S. from 2013-2018

##### Team: Edward Chen, Andres Baez

## Project Overview:
***
- Reseach question, hypothesis
  - What do we hope to achieve?
- Datasets and Background & Prior Work
  - Brief summary of our datasets
  - Previous work that have been done using these datasets
- Data Cleaning and Analysis
  - Gun Violence Data 
- Discussion and Moving Forward
- Conclusion and Implications
  - What further questions we may have
  - How to improve our study


## Research Question
***
According to Google statistics, "gun violence in the United States results in tens of thousands of deaths and injuries annually". For this research project, we are interested in visualizing just how much gun violence, in particular mass shootings (+4 victims), there is in the U.S. over the course of the last 5 years and the relationship between gun violence and criminal activity, primarily in the state or county where there is the most amount of gun violence. We are also interested in the causes of mass shootings.

## Hypothesis
***
We predict throughout the past 5 years that overall gun violence in the U.S. should be decreasing with more awareness and occurrences of mass shootings. Therefore, we also predict that crime rates, in particular the number of homicides and other criminal activity regarding the use of guns, should also decrease throughout the years.


## Datasets:
***

> Dataset Name: Gun Violence Data - Comprehensive record of over 260k US gun violence incidents from 2013-2018
 Link to Dataset: https://www.kaggle.com/jameslko/gun-violence-data/kernels
 Number of Observations: 239,677

This dataset contains an accumulation of many different mass shooting (+4 victims) within the U.S., as well as additional information regarding each case location, age demographic, gender, and house and state affiliation. We plan on using this dataset to visualize the total extent of these mass shootings in the U.S. for the past 5 years, and consequently, discover which state or county has the most amount of mass shootings.


> Dataset Name: US Mass Shootings, 1982-2018: Data From Mother Jones’ Investigation
Link to Dataset: https://www.motherjones.com/politics/2012/12/mass-shootings-mother-jones-full-data/
Number of Observations: 108

This dataset contains an aggregation of famous mass shootings and a listing of the shooting location, number of casualties, mental health details of the suspect, weapon type, race, and gender. We plan to use this dataset to find correlations between potential causes of these mass shootings. 


>Dataset Name: Crimes in Chicago - An extensive dataset of crimes in Chicago (2001-2017), by City of Chicago
Link to Dataset: https://www.kaggle.com/currie32/crimes-in-chicago
Number of Observations: 7,941,282

This dataset contains an aggregation of crimes in Chicago from 2001 to 2017. We plan to primarily use this dataset to analyze the different levels of crime in the city and potentially find any correlation between gun violence and crime rates in Chicago.


## Background and Prior Work
***

## Methods
***
Data Cleaning:

Data Analysis:

What to report:

#### Data Cleaning and Analysis per Dataset
***

In [ ]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly import __version__
print ('plotly version: ', __version__) # requires version >= 1.9.0
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.figure_factory as ff
import re
import collections

init_notebook_mode(connected=True)

import folium
from folium.plugins import HeatMap

In [ ]:
#Read in data from Gun Violence Archive
df = pd.read_csv('gv_data.csv')
print(df.describe())
print(df.head())

In [ ]:
#aggregate number of casualties
df['n_casualties'] =  df.apply(lambda row: row.n_killed + row.n_injured, axis=1)
print(df.head())

In [ ]:
#intializing global variables

#defining a color scheme
def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))
print(hex_to_rgb("#472D7BFF"))
print(hex_to_rgb("#31688EFF"))
print(hex_to_rgb("#1F9A8AFF"))
print(hex_to_rgb("#5DC863FF"))
print(hex_to_rgb("#E3E418FF"))

#defining state codes
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [ ]:
#initial cleaning on data

#clean data to remove rows where lat and lon are NaN
df = df[np.isfinite(df['latitude']) & np.isfinite(df['longitude'])]
df.head()

In [ ]:
#Find all rows where the row contains a nan value (for state, state_house_district, staet senate district)
#YOUR CODE HERE:
null_rows_state=df[df['state'].isnull()]
null_rows_state_house_district=df[df['state_house_district'].isnull()]
null_rows_state_senate_district=df[df['state_senate_district'].isnull()]


In [ ]:
#separate data by year
#calculate the number of causalties, injured, and killed 
cas_by_year = {}
cas_by_year_month = {}
for date, n, k, i in zip(df['date'], df['n_casualties'], df['n_killed'], df['n_injured']):
    year = date[:4]
    year_month = date[:7]
    if year in cas_by_year:
        cas_by_year[year][0] += n
        cas_by_year[year][1] += k
        cas_by_year[year][2] += i
    else:
        cas_by_year[year] = [n, k, i]
        
    if year_month in cas_by_year_month:
        cas_by_year_month[year_month][0] += n
        cas_by_year_month[year_month][1] += k
        cas_by_year_month[year_month][2] += i
    else:
        cas_by_year_month[year_month] = [n, k, i]

years = list(cas_by_year.keys())
year_causalties = [item[0] for item in list(cas_by_year.values())]
year_killed = [item[1] for item in list(cas_by_year.values())]
year_injured = [item[2] for item in list(cas_by_year.values())]

total_per_year = {}
for date in df['date']:
    year = date[:4]
    if year in total_per_year:
        total_per_year[year] += 1
    else:
        total_per_year[year] = 1
        
data_per_year = list(total_per_year.values())

In [ ]:
#plot total number of data points per year 
data = [go.Bar(
    x=years,
    y=data_per_year,
    name='Number Killed per Year',
    marker=dict(
        color='rgb(117,107,177)'
    ),
    opacity=0.6
)]

layout = go.Layout(
    title='Total # of Data Points per Year',
    xaxis=dict(
        title='Year',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# Data Points',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#bar chart to  see total number of casualties between years
#separate casualties by total number killed and total number injured
trace1 = go.Bar(
    x=years,
    y=year_killed,
    name='Number Killed per Year',
    marker=dict(
        color='rgb(49, 104, 142)'
    ),
    opacity=0.6
)

trace2 = go.Bar(
    x=years,
    y=year_injured,
    name='Number Injured per Year',
    marker=dict(
        color='rgb(93, 200, 99)'
    ), 
    opacity=0.6
)

data = [trace1, trace2]

layout = go.Layout(
    barmode='stack',
    title='Total # of Casualties per Year',
    xaxis=dict(
        title='Year',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# of Casualties by Killed/Injured',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#plot total number of causalties by year, split by month
#split dictionary by year 
months = []
split_by_year = []

curr_data = []
curr_year = '2013'
curr_month = '00'

for k,v in cas_by_year_month.items():
    month = k[5:]
    if month not in months:
        months.append(month)
    year = k[:4]
    if year != curr_year:
        split_by_year.append(curr_data)
        curr_data = []
        curr_data.append(v)
        curr_year = year
        curr_month = '01'
    elif month != curr_month:
        curr_data.append(v)
        curr_month = month
split_by_year.append(curr_data)

cas_year_2013 = [item[0] for item in split_by_year[0]]
cas_year_2014 = [item[0] for item in split_by_year[1]]
cas_year_2015 = [item[0] for item in split_by_year[2]]
cas_year_2016 = [item[0] for item in split_by_year[3]]
cas_year_2017 = [item[0] for item in split_by_year[4]]
cas_year_2018 = [item[0] for item in split_by_year[5]]

In [ ]:
# Create traces
trace13 = go.Scatter(
    x = months,
    y = cas_year_2013,
    mode = 'lines+markers',
    name = 'Year 2013',
    opacity = 0.75,
    marker=dict(
        color='rgb(71, 45, 123)'
    )
)
trace14 = go.Scatter(
    x = months,
    y = cas_year_2014,
    mode = 'lines+markers',
    name = 'Year 2014',
    opacity = 0.75,
    marker=dict(
        color='rgb(49, 104, 142)'
    )
)
trace15 = go.Scatter(
    x = months,
    y = cas_year_2015,
    mode = 'lines+markers',
    name = 'Year 2015',
    opacity = 0.75,
    marker=dict(
        color='rgb(31, 154, 138)'
    )
)
trace16 = go.Scatter(
    x = months,
    y = cas_year_2016,
    mode = 'lines+markers',
    name = 'Year 2016',
    opacity = 0.75,
    marker=dict(
        color='rgb(227, 228, 24)'
    )
)
trace17 = go.Scatter(
    x = months,
    y = cas_year_2017,
    mode = 'lines+markers',
    name = 'Year 2017',
    opacity = 0.75,
    marker=dict(
        color='rgb(93, 200, 99)'
    )
)
trace18 = go.Scatter(
    x = months,
    y = cas_year_2018,
    mode = 'lines+markers',
    name = 'Year 2018',
    opacity = 0.75,
    marker=dict(
        color='rgb(71, 45, 123)'
    )
)

layout = go.Layout(
    title='Total # of Casualties per Month, split by year',
    xaxis=dict(
        title='Month',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# of Casualties by Killed+Injured',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)

data = [trace13, trace14, trace15, trace16, trace17, trace18]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#explore age demographic
unique_demographic = df.participant_age_group.unique()
cleanedList = [x for x in unique_demographic if str(x) != 'nan']
demographic = []
for elem in cleanedList:
    if not elem:
        continue
    split_elem = elem.replace('|',' ').replace(':',' ').split()
    for each in split_elem:
        if len(each) >=4 and each not in demographic:
            demographic.append(each)
demographic = [x for x in demographic if not any(x1.isdigit() for x1 in x)]
print(demographic)

#count total number of casualties in each demographic
count_demographic = {}
count_demographic[demographic[0]] = 0
count_demographic[demographic[1]] = 0
count_demographic[demographic[2]] = 0

for elem in cleanedList:
    if not elem:
        continue
    split_elem = elem.replace('|',' ').replace(':',' ').split()
    for each in split_elem:
        if demographic[0] in elem:
            count_demographic[demographic[0]] += 1
        elif demographic[1] in elem:
            count_demographic[demographic[1]] += 1
        elif demographic[2] in elem:
            count_demographic[demographic[2]] += 1
print(count_demographic)
demographic_counts = list(count_demographic.values())
print(demographic_counts)
edit_demographic = ['Adult: +18', 'Teen: 12-17', 'Child: 0-11']

In [ ]:
#plot total number of casualties split by age demographic 
data = [go.Bar(
    x=edit_demographic,
    y=demographic_counts,
    name='Number of Casualties by Age Group',
    marker=dict(
        color='rgb(117,107,177)'
    ),
    opacity=0.6
)]

layout = go.Layout(
    title='Number of Casualties by Age Demographic',
    xaxis=dict(
        title='Age Demographic',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# of Casualties',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#Use same process, find demographic of shooters based on age
df_age = df['participant_age'].dropna()
#for x in df_age:
#   print(x)

unique_demographic = df_age.unique()
#create list with non null values for participant age
cleanedList = [x for x in unique_demographic if str(x) != 'nan']
ages = {}

for elem in cleanedList:
    if not elem:
        continue
    split_elem = elem.replace('|',' ').replace(':',' ').split()
    #print(split_elem)
    split_elem = split_elem[1::2]
    for x in split_elem:
        x = int(x)
        if x in ages:
            ages[x] += 1
        else:
            ages[x] = 1
            
#print(ages)
od = collections.OrderedDict(sorted(ages.items()))
above_101 = []
for k,v in od.items():
    if k > 101:
        above_101.append(k)
for x in above_101:
    del od[x]
y_axis = list(od.values())
x_axis = list(od.keys())
print(od)

In [ ]:
data = [go.Bar(
    x=x_axis,
    y=y_axis,
    name='Number of Casualties by Age',
    marker=dict(
        color='rgb(49, 104, 142)'
    ),
    opacity=0.6
)]

layout = go.Layout(
    title='Number of Casualties by Age Demographic',
    xaxis=dict(
        title='Age Demographic',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# of Casualties',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#Use same process, find gender difference of casualties
#remove missing values for gender
df_gender = df['participant_gender'].dropna()
unique_gender = df_gender.unique()

#remove any null values and store in cleaned list
cleanedList = [x for x in unique_gender if str(x) != 'nan']
genders = {}

#iterate through cleaned list and add counts to each gender
for elem in cleanedList:
    #if it is any kind of null value, continue
    if not elem:
        continue
    #split up values in cleanedList by spaces
    split_elem = elem.replace('|',' ').replace(':',' ').replace(',','').split()
    #create list with only genders
    split_elem = split_elem[1::2]

    for x in split_elem:
        if x in genders:
            genders[x] += 1
        else:
            genders[x] = 1
            
edit_demographic = ['Male', 'Female']
gender_count = list(genders.values())          

In [ ]:
#Find unique Participant Relationships
#Find total count for each unique value
data = [go.Bar(
    x=edit_demographic,
    y=gender_count,
    name='Number of Casualties by Gender',
    marker=dict(
        color='rgb(117,107,177)'
    ),
    opacity=0.6
)]

layout = go.Layout(
    title='Number of Casualties by Gender',
    xaxis=dict(
        title='Gender',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# of Casualties',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#create new bar chart separating age of casualties by gender
#create dictionary with age and number of casualties by gender
casualties_by_age_and_gender = {}
df_age_gender = df.dropna(subset = ['participant_age', 'participant_gender'])

casualties_m = {}
casualties_f ={}
for age, gender in zip(df_age_gender['participant_age'], df_age_gender['participant_gender']):
    #print(age)
    #print(gender)
    split_age = age.replace('|',' ').replace(':',' ').replace(',','').split()
    split_gender = gender.replace('|',' ').replace(':',' ').replace(',','').split()
    
    age_index = 0 
    gender_index = 0
    
    split_age_index = split_age[::2]
    split_gender_index = split_gender[::2]
    same_indices = [int(i) for i, j in zip(split_age_index, split_gender_index) if i == j]
    
    for i in same_indices:
        if 2*i+1 >= len(split_gender) or 2*i+1 >= len(split_age):
            continue
            
        actual_gender = split_gender[2*i+1]
        actual_age = split_age[2*i+1]
        if int(actual_age) > 100:
            continue
        
        if actual_gender == "Male":
            if actual_age in casualties_m:
                casualties_m[actual_age] += 1
            else:
                casualties_m[actual_age] = 1
        else:
            if actual_age in casualties_f:
                casualties_f[actual_age] += 1
            else:
                casualties_f[actual_age] = 1
            
    
    #clean the data
   
        
        
        
        
    
    #after splitting, add ages and casualties to dictionary
    
    
   # print(split_age)
   # print(split_gender)
od_m = collections.OrderedDict(sorted(casualties_m.items()))
male_count = list(od_m.values())
male_ages = list(od_m.keys())
od_f = collections.OrderedDict(sorted(casualties_f.items()))
female_count = list(od_f.values())
female_ages = list(od_f.keys())
#print(od_m)
#print(od_f)
   

In [ ]:
trace1 = go.Bar(
    x=female_ages,
    y=female_count,
    name='Number of Females Killed',
    marker=dict(
        color='rgb(49, 104, 142)'
    ),
    opacity=0.6
)

trace2 = go.Bar(
    x=male_ages,
    y=male_count,
    name='Number of Males Killed',
    marker=dict(
        color='rgb(93, 200, 99)'
    ), 
    opacity=0.6
)

data = [trace1, trace2]

layout = go.Layout(
    barmode='group',
    title='Total # of Casualties per Gender',
    xaxis=dict(
        title='Age',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='# of Casualties',
        titlefont=dict(
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
#aggregate density by state
cas_by_state = {}
for state, n, k, i in zip(df['state'], df['n_casualties'], df['n_killed'], df['n_injured']):
    if state == 'District of Columbia':
        continue
    if state in cas_by_state:
        cas_by_state[state][0] += n
        cas_by_state[state][1] += k
        cas_by_state[state][2] += i
    else:
        cas_by_state[state] = [n, k, i]
        
states = list(cas_by_state.keys())
casualties = [item[0] for item in list(cas_by_state.values())]
max_cas = max(casualties)

df_state = pd.DataFrame(states, columns=['states'])
df_state['n_casualties'] = casualties

df_state['cas_density'] = df_state.apply(lambda row: row.n_casualties/max_cas, axis=1)
df_state['code'] = df_state.apply(lambda row: us_state_abbrev[row.states], axis=1)


#get number killed and injured separated by state        
killed = [item[1] for item in list(cas_by_state.values())]
injured = [item[2] for item in list(cas_by_state.values())]

df_state['n_killed'] = killed
df_state['n_injured'] = injured
        
        
df_state['text'] = df_state.apply(lambda row: str(row.states) + '<br>' +\
                                  'Killed: '+str(row.n_killed)+
                                  ', Injured: '+str(row.n_injured), axis=1)


print(df_state.head())

In [ ]:
#choropleth map for gun casualties by state
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_state['code'],
        z = df_state['cas_density'].astype(float),
        text = df_state['text'],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Casualties ~ max 16231")
        ) ]

layout = dict(
        title = '2013-2018 US Gun Violence Casualties by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
iplot(fig)

In [ ]:
#count number of casualties > 4
len([x for x in df['n_casualties'] if x >= 4])

df_fips = df[df['n_casualties'] >= 3]
print(len(df_fips))


In [ ]:
import requests
import urllib

county_fips = []
'''
count = 0
for lat, lon, n in zip(df_fips['latitude'], df_fips['longitude'], df_fips['n_casualties']):
    if n >= 4:
        #Encode parameters 
        params = urllib.parse.urlencode({'latitude': lat, 'longitude': lon, 'format':'json'})
        #Contruct request URL
        url = 'https://geo.fcc.gov/api/census/block/find?' + params

        #Get response from API
        response = requests.get(url)

        #Parse json in response
        data = response.json()

        #Print FIPS code
        try:
            print(data['County']['FIPS'])
            county_fips.append(data['County']['FIPS'])
        except:
            county_fips.append(np.nan)
'''

#counties for 4 or more causalties per shooting
#county_fips = ['42003', '06037', '39093', '08005', '37081', '40143', '35001', '22071', '06013', '24510', '47065', '29510', '22101', '11001', '39023', '47157', '06115', '17031', '22071', '06095', '10003', '49035', '06059', '40143', '26081', '06037', '13021', '22017', '13071', '26145', '06047', '18097', '28049', '29095', '11001', '06073', '36043', '06099', '06067', '06077', '12099', '17031', '29095', '36047', '06047', '53033', '45047', '36005', '06037', '42101', '06095', '04013', '21067', '39153', '53033', '17031', '22051', '17171', '39093', '12083', '37119', '42045', '47113', '34013', '37101', '06081', '42021', '06037', '18005', '42101', '34017', '04021', '22071', '39061', '26163', '42101', '51111', '47157', '26163', '17031', '47057', '06029', '26049', '51650', '47157', '17031', '13121', '55079', '06095', '18097', '51810', '13285', '06037', '42133', '29510', '17031', '37081', '44007', '47037', '48201', '17031', '17031', '51710', '37147', '24510', '44007', '47065', '06067', '29095', '22071', '51810', '29095', '17031', '06107', '17031', '45019', '36047', '08001', '48439', '17031', '26125', '36047', '01077', '17031', '28075', '06077', '12011', '24510', '17201', '37119', '06075', '21087', '39099', '11001', '40109', '34021', '26081', '20173', '06001', '06039', '09003', '36047', '36047', '20177', '26163', '54033', '12086', '48113', '53077', '35025', '18097', '34013', '26163', '06053', '29095', '42089', '34013', '48113', '29510', '10003', '36047', '51740', '42101', '40109', '06075', '34011', '39095', '45025', '17031', '17031', '24510', '45033', '27053', '12125', '06001', '37119', '29510', '18089', '09001', '36061', '47035', '11001', '37111', '08041', '13135', '53077', '11001', '06077', '26065', '32003', '12097', '47157', '17031', '06037', '48349', '12099', '29510', '26121', '20173', '54011', '37001', '39119', '17031', '31055', '06019', '36029', '04013', '42101', '48277', '40143', '42043', '12011', '12086', '42003', '09009', '12086', '04013', '48245', '06061', '37125', '06095', '45047', '21093', '06065', '06037', '11001', '12031', '26163', '48201', '39151', '04013', '36047', '48201', '48201', '42101', '40143', '06001', '27053', '36055', '18097', '06019', '13185', '26065', '12086', '20177', '10003', '37097', '48201', '12086', '34021', '37045', '26121', '36103', '34013', '22057', '22103', '01101', '39153', '36047', '51710', '17031', '26163', '36081', '17195', '13121', '39049', '42049', '28049', '41051', '48451', '22065', '17089', '01089', '26163', '06037', '45079', '06019', '40019', '49049', '06071', '12099', '19153', '51153', '34013', '13089', '24027', '12099', '17031', '37127', '53033', '55079', '18081', '48201', '17031', '12123', '12073', '22071', '39039', '12086', '40109', '06001', '06077', '18089', '12011', '32003', '48113', '26145', '12095', '18003', '48439', '12031', '18097', '12103', '06049', '47157', '06037', '06037', '51191', '12086', '17031', '26163', '42003', '47113', '18097', '45007', '34021', '06013', '17031', '39143', '55079', '48453', '47157', '36047', '37081', '45079', '13033', '48245', '36047', '37129', '45019', '39061', '12011', '29201', '48439', '47179', '06075', '06037', '37081', '06019', '06071', '22033', '45009', '37119', '28105', '45009', '48099', '26163', '06095', '04013', '17031', '25013', '40109', '22071', '17201', '47065', '18177', '48439', '22071', '11001', '34011', '40143', '27013', '27013', '17089', '12057', '17031', '06037', '48201', '47157', '17031', '13115', '36083', '17031', '13067', '17031', '05031', '48201', '17031', '08059', '48157', '47157', '12057', '06037', '22017', '18097', '13089', '13089', '06067', '06067', '47157', '32003', '17031', '13121', '13089', '11001', '06065', '39043', '13067', '18097', '18097', '48113', '48029', '06083', '22071', '06037', '45051', '26163', '37063', '39095', '06029', '06059', '18097', '51710', '06071', '06067', '17031', '17031', '13121', '17031', '45019', '17031', '26125', '45015', '32003', '11001', '12071', '01039', '17031', '34031', '47113', '06037', '47157', '37065', '17031', '17105', '55079', '51041', '18067', '17031', '13185', '17031', '24033', '11001', '47157', '06097', '48201', '12086', '25017', '55079', '26163', '47093', '41005', '06073', '36061', '06013', '06037', '22071', '29510', '48201', '21067', '42045', '51710', '17163', '26077', '51710', '41051', '48029', '48201', '18097', '22117', '29510', '12086', '36029', '48201', '06071', '21195', '21195', '17031', '44007', '48201', '17031', '06037', '39095', '11001', '53033', '09001', '06067', '06077', '17031', '17163', '32003', '47157', '34013', '42003', '17031', '13321', '05069', '47093', '23031', '29510', '26077', '36061', '42101', '37051', '42101', '42003', '48113', '55105', '34033', '25005', '48201', '27123', '47119', '29510', '51047', '13121', '37051', '06029', '17031', '55079', '20173', '55079', '35001', '11001', '42101', '27053', '06065', '11001', '13167', '22071', '48113', '36029', '13015', '47125', '22071', '47157', '17031', '47093', '49035', '34039', '25025', '17031', '36081', '06081', '32003', '37083', '44007', '22099', '47157', '45031', '13245', '06037', '17031', '17031', '40143', '26163', '12053', '01073', '06001', '17043', '37081', '40143', '06071', '28049', '36055', '06037', '12019', '26049', '12095', '13021', '06037', '45019', '26163', '02020', '51710', '48355', '22071', '12041', '36081', '28107', '39029', '42101', '45031', '29510', '42101', '18097', '12086', '45029', '06019', '26163', '28095', '28095', '47157', '37107', '06037', '17031', '12031', '26163', '13121', '51760', '36001', '40109', '17031', '36065', '06077', '13121', '13121', '13135', '26163', '12127', '36081', '36047', '45003', '37155', '26163', '06067', '53061', '21221', '47157', '53033', '06075', '06037', '06037', '06037', '51810', '37081', '37137', '29077', '12086', '39153', '48141', '32003', '17031', '12073', '34005', '42003', '48029', '46109', '39035', '39023', '08031', '51001', '06075', '48491', '47157', '34013', '13121', '47065', '36047', '54061', '29510', '51700', '37025', '48029', '18141', '13021', '41051', '12057', '12086', '42091', '17031', '42045', '34039', '17201', '28153', '17031', '12115', '48121', '26163', '13261', '36055', '17031', '12086', '21049', '29510', '17163', '06067', '06037', '22071', '37119', '47157', '06099', '13051', '51770', '48113', '13223', '12086', '47065', '25025', '06075', '20091', '20173', '16057', '12105', '06085', '37051', '17201', '51740', '48029', '47125', '25025', '36081', '31055', '06077', '13089', '37147', '13285', '36067', '36061', '37169', '39035', '40143', '13097', '47033', '12101', '21235', '46083', '06037', '05119', '06067', '13137', '45019', '48027', '12127', '48201', '29215', '24510', '26163', '12095', '34001', '32003', '29510', '47093', '45073', '17031', '13215', '45007', '17089', '48463', '28085', '13121', '06065', '01069', '06037', '37085', '48373', '06077', '04013', '34013', '04001', '48113', '40109', '12057', '02122', '35001', '47125', '18097', '48375', '12005', '40143', '24510', '12127', '22103', '28091', '42003', '21111', '18097', '26021', '13115', '04013', '36063', '34031', '01101', '42081', '37119', '37155', '51760', '13261', '34021', '36055', '36047', '04013', '55079', '39113', '48201', '18141', '36005', '55139', '36029', '26163', '39061', '39035', '34017', '34013', '04019', '24033', '29047', '22071', '02020', '24510', '36055', '17201', '55079', '48309', '12086', '29189', '24005', '06019', '01101', '32003', '29510', '26049', '25023', '17115', '22071', '42045', '31055', '17031', '45091', '06073', '13247', '39035', '09009', '24033', '36103', '19163', '22071', '17031', '36029', '30077', '12127', '29510', '06037', '39061', '09001', '48201', '12086', '13009', '36005', '13113', '40109', '39049', '48113', '34007', '29510', '40143', '36047', '12086', '45019', '44007', '26163', '42101', '37007', '49057', '21067', '42003', '42101', '37193', '17031', '22099', '36061', '22071', '26163', '09003', '12099', '25005', '26163', '10001', '17119', '12086', '42101', '11001', '36067', '21111', '25003', '48029', '17031', '22017', '18003', '34013', '45091', '24510', '39035', '22033', '34017', '17031', '06077', '06037', '29510', '39035', '45075', '24510', '13121', '26163', '47065', '23019', '48113', '39061', '17031', '22033', '41047', '06073', '06111', '37065', '51800', '21111', '36005', '12086', '13117', '29510', '22055', '42049', '51670', '22071', '18097', '29095', '17201', '17055', '42003', '55079', '12095', '17031', '24510', '17031', '13051', '36047', '36047', '29510', '45079', '50023', '05093', '37071', '48201', '28123', '29095', '26163', '06037', '42003', '06037', '45075', '45069', '17031', '48439', '36055', '48427', '39061', '37063', '35005', '12095', '06099', '22071', '17031', '51067', '27053', '12099', '06053', '36047', '47163', '13021', '48423', '47157', '34025', '37119', '12127', '32003', '29019', '18089', '08031', '53007', '17031', '13071', '27029', '37119', '22017', '36055', '27053', '27053', '22033', '35061', '12083', '22033', '34013', '26025', '46023', '18097', '40143', '17031', '42101', '06053', '22017', '13135', '12071', '17031', '17183', '29095', '17031', '13199', '17031', '39113', '17031', '39061', '37155', '12075', '41019', '24510', '24510', '04005', '17143', '37119', '47157', '13089', '15003', '18039', '12071', '22071', '17031', '47037', '44007', '45009', '04013', '37101', '11001', '21199', '34013', '18163', '48439', '37021', '22109', '08041', '45007', '12031', '48201', '23011', '06029', '08031', '26099', '19153', '18097', '06037', '12031', '42021', '42101', '01019', '06019', '42003', '24510', '36071', '45019', '17031', '48061', '53033', '22071', '48201', '39049', '27053', '45051', '25025', '06067', '08041', '17091', '40031', '13051', '06071', '31055', '22057', '39061', '12073', '42003', '24510', '13051', '17163', '06037', '06059', '41039', '12086', '12086', '37129', '06001', '12031', '01097', '42101', '47113', '06037', '22071', '12105', '47157', '17031', '11001', '10003', '13089', '51073', '40091', '28109', '06037', '12091', '06065', '53033', '51550', '51033', '08031', '04013', '22071', '11001', '06037', '48463', '12057', '12095', '28047', '36055', '12095', '17031', '26121', '13081', '27053', '22051', '01063', '48201', '06037', '49035', '06095', '29165', '12057', '26077', '12095', '01073', '28141', '29189', '48201', '04013', '12127', '13121', '08093', '20079', '53045', '51153', '26163', '06065', '12111', '39049', '20173', '06037', '13121', '25025', '20209', '22055', '48029', '42003', '26163', '34021', '13105', '06001', '12083', '41051', '12071', '17031', '13121', '12071', '12011', '01051', '21111', '17031', '48181', '06019', '17031', '08031', '01077', '22017', '32003', '17031', '35001', '12009', '17031', '47157', '01015', '35001', '06037', '55079', '26163', '01031', '12095', '45045', '48215', '42101', '06037', '26163', '17031', '17031', '01009', '48157', '24510', '13073', '39001', '53033', '32003', '01081', '36061', '20177', '17031', '51083', '08031', '12086', '32003', '47037', '47037', '24033', '22033', '28097', '06001', '29189', '47149', '12019', '27053', '12086', '32003', '22015', '26163', '01101', '24031', '48453', '17031', '01073', '17031', '13121', '54039', '13071', '18163', '45019', '18097', '24510', '13035', '06077', '47037', '22071', '34013', '36061', '25017', '53077', '32003', '48201', '34021', '18097', '06067', '24510', '17031', '18003', '08031', '01005', '12071', '04013', '36047', '27053', '06107', '25025', '11001', '12091', '35005', '27131', '06077', '06037', '12095', '36047', '17031', '06019', '13063', '06001', '10003', '51131', '39155', '17031', '13299', '51770', '34005', '21111', '53067', '17031', '13089', '24033', '09003', '20209', '48439', '51710', '17031', '17031', '13089', '45069', '45083', '37119', '48201', '48355', '41047', '17031', '06037', '32003', '17031', '22073', '47065', '36047', '48201', '39035', '04013', '17031', '42003', '06037', '13223', '48113', '22033', '47163', '26075', '26021', '11001', '24510', '13153', '53067', '29510', '48201', '06001', '39153', '53015', '39035', '06029', '48113', '06071', '47125', '26163', '48201', '22033', '05035', '06019', '17031', '17091', '39061', '36047', '45019', '48021', '39017', '12071', '17031', '36015', '24510', '13059', '29510', '13191', '53061', '29189', '48453', '12086', '18163', '12011', '13089', '47157', '06065', '26125', '42011', '17031', '08001', '28149', '06037', '39035', '12031', '35001', '17031', '06067', '42101', '12031', '48113', '55079', '29097', '36081', '06037', '51710', '27053', '06001', '09009', '29095', '06037', '53053', '01097', '09009', '38089', '25025', '09001', '34011', '01073', '12086', '48041', '36005', '48303', '29510', '37051', '01123', '06065', '34001', '13121', '17031', '37163', '17031', '37129', '19153', '51770', '17031', '36047', '42011', '48453', '05051', '12086', '01073', '37049', '26145', '29095', '22071', '18003', '34017', '18003', '48201', '56021', '48439', '06037', '42101', '11001', '18097', '12086', '12095', '42003', '42101', '51680', '48201', '47037', '53057', '24510', '17019', '37067', '48201', '48201', '06087', '45007', '06075', '06019', '17091', '48029', '27053', '22071', '36047', '26081', '06067', '17031', '48113', '17031', '01097', '06037', '17201', '17031', '06075', '01089', '06001', '40081', '51770', '39113', '13151', '12086', '28049', '39113', '06065', '06037', '39095', '24033', '36071', '36061', '22017', '47157', '05119', '34013', '17031', '42101', '06001', '17031', '12011', '48029', '45051', '17031', '47157', '53033', '48201', '42125', '48201', '16001', '09001', '29095', '06067', '06073', '48029', '12031', '12095', '47045', '17031', '06037', '12051', '28033', '36001', '21111', '02090', '22071', '17031', '22071', '29095', '06037', '48215', '06037', '24510', '06033', '34013', '06037', '18039', '35001', '21067', '48201', '06019', '36047', '37065', '12095', '24510', '06067', '22055', '37119', '17031', '17031', '37119', '17031', '48029', '37187', '01073', '17031', '37195', '37157', '36119', '17031', '01045', '13215', '39139', '01101', '09009', '48201', '13277', '12086', '28011', '48113', '45063', '17201', '41047', '48085', '06071', '06037', '12011', '26049', '21205', '08041', '17031', '34013', '06053', '29095', '21235', '28113', '17031', '08021', '48201', '12086', '28107', '06019', '47157', '12099', '13063', '13121', '48039', '17031', '48029', '17031', '13095', '53035', '47075', '48215', '04013', '24033', '22017', '27053', '47157', '11001', '12099', '28163', '42003', '40111', '05119', '45079', '39035', '28049', '22071', '47071', '17031', '36071', '12079', '29155', '06053', '17031', '36007', '27123', '42101', '18141', '51710', '16041', '19169', '55025', '39035', '06037', '28075', '39035', '36055', '20079', '12031', '39155', '18089', '51700', '39061', '06065', '42101', '48165', '20177', '06067', '06025', '44007', '13215', '17031', '27053', '17089', '22071', '17031', '24005', '22051', '48453', '26163', '21111', '48439', '55073', '22001', '24017', '29510', '06019', '42043', '39035', '26163', '42077', '47093', '39035', '39061', '48201', '12117', '39151', '17031', '12057', '40109', '06013', '17031', '19153', '48303', '42101', '22019', '27123', '17031', '48201', '37035', '18003', '48029', '06071', '13121', '18097', '17195', '06065', '01073', '17031', '42101', '06095', '37139', '17201', '39049', '53033', '21111', '12009', '22071', '13157', '12086', '34013', '42045', '09003', '06037', '51810', '24031', '06037', '06067', '06037', '20177', '06073', '12031', '45051', '48113', '12031', '26049', '17031', '20209', '48201', '34021', '39089', '17031', '05031', '12055', '48439', '45029', '42101', '04013', '19153', '06015', '12086', '11001', '36061', '28085', '01113', '28059', '34031', '21111', '39017', '29510', '48479', '06037', '29510', '22071', '22071', '18175', '12095', '06019', '49035', '42131', '22071', '48439', '47157', '47065', '48201', '17031', '29189', '24510', '24510', '06075', '39151', '51510', '35039', '51760', '36047', '17031', '39049', '11001', '29510', '17031', '45051', '01097', '13121', '37147', '36061', '12011', '06073', '34005', '29510', '11001', '17031', '12086', '39153', '36005', '34013', '17091', '37067', '05119', '06001', '34017', '45047', '26125', '01001', '51650', '12095', '39071', '37183', '13051', '23025', '17031', '28075', '42017', '39061', '17119', '51025', '06065', '01073', '45077', '17031', '42101', '55079', '34021', '04013', '06037', '39049', '48201', '51760', '39061', '22033', '17031', '12031', '37081', '17031', '10003', '36029', '37159', '48141', '17115', '08041', '17031', '06019', '53077', '24510', '18141', '11001', '12091', '30003', '12073', '45005', '34029', '06001', '36005', '36005', '17031', '29189', '42101', '06085', '06067', '42077', '51800', '06037', '29510', '36103', '48029', '39035', '17031', '36047', '34013', '24510', '34013', '47157', '37083', '17031', '06071', '29189', '06067', '18163', '35009', '06067', '17031', '17031', '21111', '06037', '17031', '17031', '01047', '48085', '22071', '47157', '42101', '42133', '53063', '45079', '51730', '26163', '18089', '17183', '06029', '09003', '09009', '24510', '47037', '36067', '47157', '42101', '24510', '22071', '06037', '47157', '20045', '12115', '12086', '06001', '04021', '17031', '48439', '04013', '29019', '06075', '06085', '39087', '06059', '04013', '48201', '28075', '36047', '24025', '08069', '42045', '12057', '47037', '12031', '09003', '01017', '06075', '21111', '39095', '51730', '48453', '18089', '27053', '36029', '06037', '22103', '06029', '39099', '48453', '06037', '48493', '48113', '49049', '42101', '39113', '04013', '48113', '22071', '37051', '18089', '06019', '37107', '13121', '06103', '06013', '13215', '22045', '27053', '42033', '39035', '28085', '26163', '26163', '34017', '29510', '47157', '42003', '17031', '37119', '25025', '06077', '48113', '21111', '39049', '06037', '28007', '26125', '13215', '22055', '42101', '22097', '01073', '28149', '01109', '04013', '12086', '35001', '47157', '42003', '32003', '08035', '17031', '34025', '01089', '05095', '17031', '28035', '06037', '01011', '06089', '12011', '48167', '29169', '37119', '47037', '01089', '01063', '12001', '45091', '11001', '01015', '42043', '17031', '12095', '39097', '42101', '21157', '13093', '11001', '12083', '06037', '21227', '42051', '42011', '18097', '29510', '39035', '08041', '12099', '13151', '21115', '26163', '22071', '12011', '26125', '20209', '47157', '48029', '04013', '12099', '26163', '09001', '36047', '12086', '17201', '29083', '01113', '06055', '37007', '18141', '26145', '17019', '04013', '06099', '12003', '53053', '36047', '55079', '17031', '21111', '17021', '06075', '49035', '48085', '34007', '34025']

#counties for 3 or more causalties per shooting
county_fips = ['42003', '06037', '39093', '08005', '37081', '40143', '35001', '22071', '06013', '24510', '47065', '29510', '22101', '11001', '39023', '47157', '06115', '17031', '22071', '06095', '10003', '49035', '06059', '40143', '26081', '06037', '13021', '22017', '13071', '26145', '06047', '18097', '28049', '29095', '11001', '06073', '36043', '06099', '06067', '06077', '12099', '17031', '29095', '36047', '06047', '53033', '45047', '36005', '06037', '42101', '06095', '04013', '21067', '39153', '53033', '17031', '22051', '17171', '39093', '12083', '37119', '42045', '47113', '34013', '37101', '06081', '42021', '06037', '18005', '42101', '34017', '04021', '22071', '39061', '26163', '42101', '51111', '47157', '26163', '17031', '47057', '06029', '26049', '51650', '47157', '17031', '13121', '55079', '06095', '18097', '51810', '13285', '06037', '42133', '29510', '17031', '37081', '44007', '47037', '48201', '17031', '17031', '51710', '37147', '24510', '44007', '47065', '06067', '29095', '22071', '51810', '29095', '17031', '06107', '17031', '45019', '36047', '08001', '48439', '17031', '26125', '36047', '01077', '17031', '28075', '06077', '12011', '24510', '17201', '37119', '06075', '21087', '39099', '11001', '40109', '34021', '26081', '20173', '06001', '06039', '09003', '36047', '36047', '20177', '26163', '54033', '12086', '48113', '53077', '35025', '18097', '34013', '26163', '06053', '29095', '42089', '34013', '48113', '29510', '10003', '36047', '51740', '42101', '40109', '06075', '34011', '39095', '45025', '17031', '17031', '24510', '45033', '27053', '12125', '06001', '37119', '29510', '18089', '09001', '36061', '47035', '11001', '37111', '08041', '13135', '53077', '11001', '06077', '26065', '32003', '12097', '47157', '17031', '06037', '48349', '12099', '29510', '26121', '20173', '54011', '37001', '39119', '17031', '31055', '06019', '36029', '04013', '42101', '48277', '40143', '42043', '12011', '12086', '42003', '09009', '12086', '04013', '48245', '06061', '37125', '06095', '45047', '21093', '06065', '06037', '11001', '12031', '26163', '48201', '39151', '04013', '36047', '48201', '48201', '42101', '40143', '06001', '27053', '36055', '18097', '06019', '13185', '26065', '12086', '20177', '10003', '37097', '48201', '12086', '34021', '37045', '26121', '36103', '34013', '22057', '22103', '01101', '39153', '36047', '01125', '04013', '12086', '51710', '17031', '13115', '18003', '26163', '26163', '36047', '12091', '36081', '48439', '17195', '13121', '12031', '36055', '48201', '06099', '22033', '39049', '12095', '42101', '42049', '51107', '47157', '12086', '45075', '39049', '12086', '37183', '12099', '48451', '28049', '55025', '34031', '41051', '48451', '22065', '13021', '17089', '01089', '48113', '12099', '12031', '26163', '29095', '49011', '06037', '47113', '20009', '45079', '18039', '06075', '12021', '06019', '40019', '49049', '12111', '06071', '12099', '26163', '36067', '17031', '05069', '45079', '42049', '21015', '55109', '48157', '19153', '27053', '48339', '12021', '08059', '06029', '51153', '34013', '13089', '36009', '34017', '48027', '24027', '19193', '12071', '12099', '17031', '40109', '01083', '06037', '06019', '37127', '39007', '53033', '48029', '22017', '55079', '49049', '19113', '17031', '12031', '01101', '24005', '17031', '20173', '12086', '18081', '48201', '12086', '12099', '47093', '47093', '45083', '17031', '12123', '12073', '13045', '22071', '39039', '08001', '12086', '39061', '40109', '12086', '39095', '22097', '06059', '17031', '06037', '05119', '26163', '37083', '18117', '22097', '06001', '47107', '37025', '06077', '18089', '05107', '42101', '17161', '12011', '32003', '40125', '48113', '26145', '12095', '18003', '48439', '53029', '12031', '53061', '12031', '06095', '06037', '17031', '12086', '51760', '36005', '39113', '18097', '12103', '06049', '37063', '05145', '17031', '37105', '47157', '17031', '24033', '06037', '06037', '06053', '08001', '01073', '01073', '36055', '27041', '05091', '51191', '12086', '17031', '26163', '34023', '45079', '42003', '37051', '47113', '40079', '24031', '34013', '01097', '18097', '45007', '25027', '26163', '13245', '34021', '06013', '08041', '08041', '17031', '39143', '55079', '48453', '47157', '18089', '02020', '42101', '01073', '12031', '37023', '47157', '47157', '22071', '06065', '36047', '36005', '06067', '06067', '13285', '17031', '51710', '45079', '06067', '39049', '37081', '45079', '13033', '48245', '12095', '36047', '12095', '08041', '12086', '06047', '48355', '37129', '45019', '45091', '39061', '12095', '12011', '09009', '29201', '17031', '37159', '01097', '48439', '17031', '47179', '06075', '06037', '06037', '37081', '13077', '24510', '37051', '18141', '06019', '22033', '13021', '36085', '06071', '22033', '49035', '37049', '45009', '37081', '37191', '28059', '37119', '28027', '28105', '18097', '45009', '18097', '36029', '19097', '51760', '17031', '37147', '48099', '26163', '01089', '06095', '04013', '17031', '50021', '48183', '29510', '36047', '25013', '40109', '51700', '09003', '17031', '48113', '22071', '17031', '01073', '05051', '17201', '34021', '37183', '47065', '45013', '34013', '45013', '18177', '17031', '37129', '17031', '26125', '48439', '06037', '39035', '12103', '13147', '17031', '22071', '12011', '13147', '47079', '06071', '11001', '12086', '34011', '22033', '41051', '28081', '36029', '01073', '47037', '40143', '04019', '01061', '27013', '05119', '27013', '17089', '01073', '12057', '17031', '18097', '34021', '06037', '48113', '37125', '17031', '51650', '48201', '17031', '05119', '39025', '13089', '47157', '01097', '47125', '54039', '17031', '06059', '13115', '37183', '29510', '36083', '24510', '02020', '48201', '22071', '06037', '34025', '17031', '18177', '13067', '36047', '05051', '54039', '17031', '17031', '06037', '33011', '42101', '12086', '42101', '05031', '17031', '48201', '39061', '17097', '39035', '26163', '26115', '17031', '45019', '08059', '06007', '06019', '48157', '47157', '45019', '11001', '21047', '12057', '16001', '39035', '17031', '06029', '12095', '06037', '22017', '12095', '18097', '13089', '48085', '13089', '06067', '06067', '47157', '28049', '32003', '17031', '13121', '06037', '06099', '13089', '06071', '06007', '11001', '36063', '06065', '25027', '22071', '55079', '39043', '17031', '13067', '12099', '26125', '18097', '09009', '37119', '13121', '17143', '18097', '48113', '24510', '24510', '22071', '48029', '06083', '22071', '06037', '29510', '48135', '29189', '45049', '45051', '12071', '12005', '26163', '13015', '17031', '37063', '06013', '26163', '39095', '22071', '12105', '06029', '17111', '17031', '22071', '55079', '17031', '36047', '39035', '06059', '06107', '22079', '20177', '06037', '48439', '13067', '17073', '17031', '17031', '06047', '13089', '18097', '01101', '51710', '06047', '31055', '47157', '36047', '20177', '34001', '22073', '06071', '17143', '06081', '06067', '17031', '51810', '48439', '17031', '08031', '40143', '48439', '01097', '26163', '51089', '48439', '13121', '18097', '48441', '17031', '45015', '53033', '24043', '45019', '37001', '06029', '17031', '48441', '26125', '39061', '06001', '45015', '06073', '06073', '32003', '36047', '11001', '26163', '12071', '01039', '22033', '17031', '18003', '04013', '34031', '53011', '53011', '41051', '10003', '10003', '49047', '04013', '45085', '39049', '53033', '47113', '22073', '17031', '36119', '06037', '47157', '37065', '17031', '17105', '17179', '55079', '51041', '29510', '18067', '17031', '11001', '48141', '18097', '17031', '22055', '42003', '13185', '17031', '12105', '12001', '06065', '34033', '48127', '48201', '08059', '32003', '24033', '28075', '11001', '47157', '17183', '45021', '34017', '36005', '06097', '36029', '47125', '18117', '20173', '48157', '48201', '12086', '25017', '48029', '55079', '01073', '26163', '47093', '41005', '42045', '06073', '13089', '42045', '36061', '06013', '29095', '51087', '24510', '06037', '48201', '22071', '29510', '17031', '24510', '36047', '48201', '36047', '47065', '21067', '34031', '26163', '13089', '40065', '29189', '17031', '17031', '42045', '29189', '17031', '48029', '29217', '51710', '17163', '26077', '40143', '08059', '36055', '51710', '41051', '48029', '27053', '48201', '18097', '24510', '22117', '29510', '12086', '36029', '48201', '06069', '09001', '06071', '21195', '21195', '06095', '22071', '17031', '19163', '40143', '44007', '42017', '22071', '20091', '20091', '48201', '27053', '28049', '06097', '17031', '48091', '06037', '17031', '39095', '45021', '29047', '11001', '53033', '11001', '09001', '06067', '49005', '48201', '36061', '48043', '34017', '06077', '06037', '06037', '47037', '17031', '35001', '17031', '36047', '17031', '39073', '28093', '21223', '12127', '01101', '37067', '17163', '32003', '29510', '29095', '42073', '17161', '45071', '47157', '34013', '29510', '48215', '06065', '48215', '24510', '28049', '36029', '42003', '42045', '17031', '13321', '39035', '06081', '05069', '06081', '47093', '42045', '23031', '47037', '11001', '42003', '36005', '13051', '29510', '22033', '18089', '26077', '21195', '39061', '21081', '36061', '42101', '37085', '29510', '01073', '48201', '05119', '29095', '22071', '32003', '17031', '06073', '37051', '21131', '42101', '45045', '17031', '36067', '42101', '24510', '47069', '42003', '48113', '04017', '55105', '34033', '25005', '04013', '17031', '17031', '48439', '48201', '27123', '47119', '27053', '29510', '48427', '51047', '36005', '37083', '12099', '13121', '22045', '12099', '12086', '37051', '18181', '12095', '47065', '06047', '13153', '06029', '13135', '17031', '55079', '20173', '47157', '48201', '09009', '55079', '35001', '12095', '11001', '11001', '42101', '27053', '39113', '36047', '19153', '18019', '06065', '11001', '13167', '22071', '29095', '48113', '34001', '10003', '55079', '36029', '06059', '06037', '13015', '48027', '47125', '22071', '17031', '06047', '55079', '47157', '51700', '13185', '45007', '47157', '26163', '17031', '24033', '06037', '06053', '47093', '47157', '36061', '49035', '34039', '21159', '13059', '31055', '06065', '37119', '25025', '17031', '36005', '39049', '36081', '41051', '06081', '32003', '17031', '37083', '44007', '22099', '17031', '39095', '17031', '12095', '41011', '48201', '05051', '34013', '47157', '26163', '17031', '12095', '55025', '45031', '45057', '36029', '45057', '20209', '26099', '13223', '51177', '13245', '26099', '06037', '17031', '06037', '06037', '17031', '18085', '17031', '54081', '40143', '26163', '20209', '34007', '31055', '31055', '12053', '26163', '01073', '36047', '37015', '06001', '25025', '17043', '37081', '42133', '04019', '40143', '29095', '06071', '28049', '36055', '06037', '53033', '34003', '39023', '06047', '18097', '37003', '12019', '26049', '17089', '01073', '12095', '17031', '17031', '13021', '06037', '17031', '47065', '06037', '39035', '42101', '45019', '45019', '26163', '24043', '22051', '13121', '29510', '26163', '21083', '06077', '13207', '06077', '02020', '51710', '12099', '22071', '48355', '22071', '39095', '01073', '27119', '17031', '12041', '17031', '17031', '36081', '06065', '17031', '06065', '17031', '39049', '28107', '47037', '32003', '06087', '39029', '42101', '42101', '17031', '05051', '13077', '01073', '39151', '27053', '06037', '55079', '22019', '17031', '22019', '06071', '17031', '45031', '34013', '29510', '17031', '29510', '17031', '47037', '17031', '11001', '12105', '17031', '42101', '18157', '18097', '12086', '26081', '45029', '06019', '26163', '36047', '42101', '36061', '22033', '28095', '28095', '06037', '11001', '18097', '47157', '37107', '24510', '06037', '17031', '29510', '12031', '29510', '36005', '42101', '24005', '26163', '11001', '06001', '48113', '13063', '29510', '47157', '12086', '17031', '42101', '22071', '13121', '51760', '36001', '40109', '17031', '06087', '42003', '53033', '45019', '17031', '36065', '06001', '25027', '29510', '36029', '34013', '51690', '06077', '37105', '13121', '13121', '29155', '13135', '40137', '13135', '01073', '27053', '13121', '18003', '26163', '39049', '06037', '51760', '17031', '12127', '06065', '36081', '36047', '17031', '39061', '45003', '51760', '05111', '12099', '37155', '06013', '12015', '12083', '11001', '17031', '48149', '26163', '17031', '17031', '05051', '06067', '53061', '12099', '48113', '24033', '21221', '04013', '06037', '51760', '39035', '53033', '13121', '47157', '36081', '11001', '51710', '06037', '53033', '29189', '06071', '06001', '47157', '06013', '06075', '29510', '40109', '06037', '48201', '06037', '17031', '06037', '51810', '39095', '32031', '37081', '12117', '12099', '12085', '13245', '34013', '06071', '28049', '08123', '01069', '26163', '04013', '37137', '18003', '37119', '16083', '29077', '12033', '12086', '45033', '13121', '39153', '48141', '17031', '06037', '42003', '37123', '53053', '32003', '53021', '53053', '53021', '17031', '17031', '12073', '34005', '18097', '48453', '42003', '06007', '29510', '50005', '48029', '48029', '12073', '28149', '06109', '46109', '39035', '39023', '51810', '34031', '08031', '48201', '51001', '51550', '06037', '48201', '39095', '06075', '29189', '48491', '41067', '51710', '29510', '17031', '47157', '34013', '48201', '13121', '31055', '12119', '47065', '06037', '34013', '37155', '37155', '42101', '36047', '26145', '41039', '54061', '26163', '40109', '13051', '29510', '51700', '17197', '26049', '37025', '42101', '12081', '29510', '01097', '06065', '45005', '12057', '48029', '18141', '08001', '28059', '13051', '48245', '48439', '06013', '13021', '13089', '09009', '41051', '17031', '06037', '12057', '29189', '12086', '36047', '42091', '17031', '48439', '42045', '34039', '45019', '06037', '05119', '05119', '51153', '29095', '36047', '17201', '37147', '31055', '28153', '17031', '12115', '48121', '26163', '13261', '36055', '27123', '17031', '12086', '39135', '32003', '37135', '28027', '21049', '29510', '12021', '36047', '25025', '12086', '40143', '22017', '17163', '37155', '37155', '42003', '37081', '01109', '01073', '06067', '48135', '06037', '39061', '48439', '17031', '11001', '48355', '22071', '42003', '37119', '37063', '04013', '06037', '37119', '06071', '18089', '47157', '53033', '34017', '39093', '54025', '06099', '48201', '13051', '55079', '40101', '01097', '42099', '34007', '06037', '17031', '06065', '42071', '51770', '48113', '36005', '47183', '06019', '13223', '12086', '17031', '47065', '25025', '48439', '48201', '47157', '42089', '06075', '06029', '20091', '32003', '42101', '20173', '13089', '16057', '12105', '42101', '40109', '06085', '01087', '37051', '17201', '12086', '06013', '51740', '12107', '11001', '12105', '29165', '06001', '12009', '17031', '47119', '12095', '34031', '27037', '01097', '42101', '48029', '06001', '55105', '42003', '54037', '11001', '39113', '47125', '06059', '06073', '10003', '25025', '36081', '06099', '31055', '24033', '26163', '27053', '04011', '06077', '22071', '01055', '13089', '37119', '39113', '37147', '13185', '12057', '12073', '13285', '48113', '36067', '45019', '36061', '35001', '12057', '37129', '53075', '48309', '48355', '37169', '39035', '25001', '37129', '47125', '40143', '54033', '02020', '42003', '21111', '13097', '42101', '06071', '45019', '47033', '42003', '20045', '12101', '40099', '48201', '37063', '18117', '21235', '46083', '53061', '29077', '35001', '35001', '51179', '36103', '01019', '06037', '05119', '47037', '48355', '48355', '05119', '29043', '06067', '36119', '06065', '12057', '13121', '13137', '45019', '53033', '48027', '13121', '12127', '12127', '37119', '32003', '55079', '48201', '17031', '54029', '29215', '47157', '06077', '01073', '29019', '24510', '39035', '37065', '28075', '15009', '26163', '12095', '06013', '34001', '06059', '37063', '17031', '22071', '32003', '48439', '12105', '13121', '48029', '20173', '06007', '06037', '29510', '29095', '47093', '12057', '12057', '51760', '45073', '02020', '12095', '34011', '48479', '17031', '06001', '26005', '16001', '13215', '45007', '48113', '29510', '17089', '05143', '12031', '48463', '28085', '01055', '41051', '39035', '36067', '13121', '47157', '29095', '48029', '06065', '47157', '01069', '13059', '06037', '17031', '37085', '48245', '29077', '42101', '17031', '48373', '06077', '29510', '04013', '27123', '34013', '24043', '04001', '45013', '48201', '48201', '48113', '40109', '36093', '47157', '06023', '39035', '12057', '06023', '12057', '04025', '02122', '37063', '35001', '55079', '47125', '17031', '18097', '12097', '40005', '48375', '42101', '12057', '12005', '29510', '25025', '51700', '47037', '48113', '53035', '33011', '06077', '48331', '06065', '29510', '22005', '12086', '13089', '12023', '06037', '39049', '36047', '53025', '55079', '40143', '24510', '06019', '45079', '12127', '22103', '37041', '29510', '28091', '40123', '55079', '34007', '42003', '21111', '18097', '37051', '25013', '39035', '26021', '09003', '34007', '51700', '13115', '45007', '06067', '41051', '37081', '24005', '12095', '18097', '39035', '47157', '51153', '01081', '12086', '13051', '22071', '55079', '06001', '04013', '47027', '48215', '12086', '06037', '04013', '26153', '53053', '30047', '17031', '36063', '34031', '01101', '01051', '42081', '10001', '36005', '29095', '37119', '37155', '37045', '12031', '32003', '06037', '42133', '09009', '17031', '17031', '48439', '51760', '17031', '13261', '12031', '40023', '13313', '40023', '12031', '12095', '17031', '39061', '42081', '34021', '41051', '36055', '36029', '51810', '36047', '12011', '04013', '29189', '06065', '36001', '42101', '55079', '17031', '24033', '06073', '45085', '06053', '39113', '48201', '18141', '36005', '06019', '55139', '48041', '48113', '24033', '01073', '36029', '51700', '22051', '06029', '24510', '36001', '02050', '29510', '30063', '39061', '29189', '29510', '42101', '26163', '39061', '51760', '51810', '26163', '39095', '39165', '39061', '39035', '48491', '34017', '36029', '34013', '17031', '04019', '26125', '22017', '34013', '24033', '29047', '22071', '29019', '17031', '02020', '22071', '36119', '17031', '17201', '25009', '37119', '39061', '29510', '24510', '36055', '17201', '55079', '06037', '42099', '48309', '37147', '39153', '01097', '12103', '08001', '12086', '12031', '40143', '29189', '37071', '12057', '24005', '47151', '06107', '06037', '12095', '24510', '26049', '17031', '06019', '36061', '55079', '12095', '04001', '48245', '06001', '01101', '13285', '18141', '29095', '17031', '36081', '32003', '29510', '26049', '51730', '25023', '17115', '06077', '27053', '24510', '36029', '53063', '06073', '38035', '42101', '06037', '22071', '45047', '42011', '41051', '42045', '31055', '17031', '17031', '55079', '42133', '08005', '45091', '17031', '12086', '36103', '06073', '12031', '29095', '17031', '36081', '48113', '10003', '39113', '13247', '45035', '39035', '09009', '24033', '12095', '47031', '42007', '36103', '19163', '22071', '17031', '36029', '28079', '12081', '11001', '36061', '21067', '49035', '37051', '30077', '17031', '12127', '21067', '29510', '04013', '06037', '34029', '42003', '36047', '39061', '09001', '13021', '48201', '37155', '17031', '48063', '12086', '13009', '51800', '51590', '36005', '22099', '13113', '40109', '17143', '32003', '39049', '48113', '34007', '29510', '40143', '45051', '36055', '17031', '36047', '25025', '12086', '17031', '45019', '06013', '17031', '18089', '17031', '12011', '39061', '06001', '44007', '26163', '34031', '13201', '42101', '17115', '37007', '22051', '12099', '47157', '49057', '21067', '42003', '26145', '48113', '06067', '34017', '35017', '26163', '42101', '12011', '48209', '37193', '17031', '17031', '17031', '22099', '17163', '36061', '02090', '22071', '47093', '26163', '06001', '09003', '12099', '26163', '26049', '25005', '01073', '26163', '10001', '22071', '17119', '12086', '36061', '53061', '19061', '23015', '24510', '17031', '06103', '48051', '26081', '42101', '18095', '11001', '36067', '06019', '21111', '23015', '25003', '36067', '47157', '17031', '48029', '17031', '22017', '39017', '12099', '18003', '12086', '34013', '17031', '45091', '01103', '27053', '27053', '06037', '48157', '45019', '24510', '39035', '42101', '27053', '47065', '17031', '29510', '06037', '18089', '06067', '18095', '37125', '26163', '48439', '22033', '12086', '34017', '17031', '48139', '06077', '06037', '48201', '06067', '12069', '29510', '39035', '53011', '17031', '39035', '45075', '24510', '13121', '26163', '47065', '23019', '48113', '48113', '48201', '53073', '24005', '37155', '11001', '39035', '39061', '17031', '17031', '22033', '41047', '06037', '06077', '06073', '06111', '22051', '25027', '37065', '51800', '08031', '21111', '08031', '48439', '22071', '13121', '36005', '39113', '22057', '32027', '12086', '22051', '25017', '24510', '13117', '01047', '24510', '29510', '06037', '06095', '36005', '22055', '42049', '45015', '45083', '06037', '45057', '55105', '17031', '51670', '22071', '18097', '17031', '26163', '29095', '22033', '35001', '17201', '26163', '25025', '17055', '30003', '11001', '42003', '36047', '53033', '29095', '08005', '19013', '06075', '47107', '55079', '39049', '36047', '17031', '12095', '36005', '17031', '24510', '22033', '45075', '17031', '17031', '13051', '26077', '36047', '29510', '36047', '25025', '13063', '29510', '18097', '36067', '12086', '13121', '36055', '40125', '45079', '13067', '50023', '42101', '20169', '37097', '41051', '26049', '49035', '05093', '26081', '37071', '48201', '42101', '06059', '53033', '48027', '12099', '20057', '36119', '28123', '06001', '29095', '26163', '20173', '06037', '10003', '26163', '06037', '11001', '18089', '12071', '48135', '29510', '25025', '25025', '29189', '36071', '36047', '24510', '17031', '55079', '17031', '42003', '06037', '12095', '45075', '45045', '51680', '41051', '22085', '42101', '34035', '26077', '55105', '37107', '06001', '45069', '17031', '17031', '34007', '48439', '17031', '12057', '09011', '27053', '11001', '36055', '39049', '13121', '48427', '39061', '37063', '17031', '39035', '36083', '48201', '35005', '12095', '53033', '39035', '18097', '28067', '36047', '37119', '17031', '06099', '42101', '47157', '34021', '17031', '13221', '22071', '17031', '42003', '17031', '51067', '27053', '06037', '12099', '18065', '06053', '11001', '18097', '36047', '13135', '06001', '47163', '13021', '25025', '47157', '48423', '51053', '47157', '29095', '17031', '39061', '27109', '37081', '39153', '34025', '29510', '24510', '39095', '17031', '17031', '22071', '37159', '17031', '36005', '26163', '06067', '17031', '17031', '11001', '51730', '17031', '06037', '37119', '37119', '17031', '06037', '12127', '32003', '29019', '51161', '28059', '47037', '17031', '18089', '08031', '29189', '35001', '53007', '29095', '17031', '13071', '24015', '24033', '22071', '27029', '42003', '01077', '17197', '37119', '22017', '36055', '27053', '27053', '55079', '42003', '13051', '26163', '51153', '48113', '22033', '51770', '35061', '48113', '12083', '17031', '06017', '22033', '17031', '28011', '34013', '01073', '17031', '42003', '42003', '32003', '17031', '12127', '26025', '46023', '06065', '49035', '17031', '13069', '09005', '37119', '28151', '45019', '18097', '39103', '29095', '53077', '36047', '29095', '06037', '06059', '12099', '40143', '17031', '42101', '17031', '01047', '39093', '06053', '18091', '08031', '37083', '45007', '01089', '48025', '22017', '17197', '13135', '06037', '12071', '17031', '05051', '48201', '17183', '47037', '48201', '12011', '11001', '39035', '17031', '29095', '17031', '13199', '29069', '19129', '34013', '17031', '39113', '17031', '39061', '37155', '42101', '13115', '13051', '06053', '29095', '18089', '13121', '12075', '41019', '40143', '24510', '32003', '41051', '11001', '22071', '47179', '06071', '42045', '48113', '12011', '22095', '37063', '24510', '36047', '01073', '34039', '01073', '04005', '13215', '17143', '42097', '12011', '31055', '12095', '39113', '12023', '37119', '47157', '28001', '17031', '17031', '04013', '18097', '45045', '36061', '12086', '13089', '17031', '37157', '10003', '47037', '13013', '51710', '21111', '15003', '47139', '10001', '40017', '18039', '42133', '12071', '44007', '12086', '09003', '06059', '22071', '18167', '06077', '17031', '37065', '06009', '06047', '39061', '53067', '39151', '20173', '10003', '13051', '29510', '47037', '35033', '06065', '44007', '13227', '12071', '45009', '04013', '37101', '11001', '45025', '21199', '12111', '34013', '18163', '21151', '22017', '13051', '48439', '37021', '18097', '17031', '22109', '54011', '17031', '29189', '47101', '37057', '48201', '51710', '06001', '17031', '45013', '08041', '36061', '39049', '06073', '45051', '06013', '45007', '31055', '17031', '20205', '12031', '12127', '12011', '42091', '48201', '23011', '48201', '42049', '09001', '34039', '17031', '36055', '17031', '06029', '37001', '08031', '13299', '12095', '26099', '19153', '04013', '40109', '39061', '21237', '36061', '18097', '17031', '06037', '06037', '12086', '24033', '12031', '12086', '11001', '47157', '42021', '28035', '17031', '37157', '13021', '42101', '39099', '01019', '26163', '22051', '51760', '36029', '48201', '42011', '51059', '06019', '21047', '06083', '42003', '24510', '29175', '29175', '06013', '19153', '36071', '45019', '17031', '27053', '48061', '53033', '22071', '32031', '45007', '48201', '05069', '39049', '27053', '06013', '45051', '25025', '26103', '01101', '48029', '06067', '34013', '11001', '08041', '42091', '17091', '40031', '48157', '17031', '13051', '48013', '06071', '36047', '26163', '11001', '12031', '37119', '06071', '39119', '48201', '13057', '37067', '32031', '31055', '17031', '22057', '39061', '28153', '17031', '51760', '32003', '12073', '40143', '47037', '27123', '51680', '25025', '42101', '40113', '51590', '42003', '53005', '24510', '13051', '17163', '36047', '37107', '06037', '42003', '06059', '06037', '42101', '13215', '29095', '17031', '04017', '22105', '41039', '22051', '26163', '06095', '47097', '13051', '39035', '24033', '41047', '06029', '12086', '17197', '51760', '34031', '39035', '12086', '12086', '37129', '06001', '18089', '40109', '37063', '42101', '06001', '47037', '51550', '48113', '42003', '04019', '48135', '42101', '55003', '48113', '34003', '12031', '17097', '01097', '37147', '37147', '42101', '47113', '01073', '06029', '37117', '35001', '22101', '24510', '31055', '10003', '06037', '22071', '18089', '12011', '48201', '51590', '12057', '06037', '06107', '20173', '17031', '06071', '40047', '37083', '42101', '01127', '17031', '12031', '47157', '12095', '06073', '01113', '29095', '17031', '36061', '12105', '17031', '47157', '17031', '17031', '11001', '06019', '37183', '36047', '51650', '28049', '05043', '17031', '10003', '17031', '39061', '06037', '28049', '13089', '13089', '51700', '39113', '47037', '42007', '17031', '29097', '49035', '51073', '12031', '13285', '51610', '17031', '17031', '17031', '48029', '40091', '13121', '28109', '06037', '09001', '18097', '12091', '08041', '06065', '36103', '37083', '48061', '53033', '25021', '51550', '04021', '55079', '06037', '12086', '42045', '32003', '17031', '51033', '37165', '06037', '06019', '12086', '08031', '40109', '06037', '37051', '45045', '04013', '22071', '51710', '48157', '53033', '12101', '12009', '11001', '36005', '06107', '06037', '01063', '25017', '24510', '48463', '12057', '12095', '28047', '06041', '36055', '12095', '17031', '26121', '06037', '36001', '24025', '01093', '32003', '51760', '13081', '27053', '31055', '48265', '17031', '12127', '21111', '22051', '01063', '06075', '48201', '06001', '22033', '06037', '06053', '18011', '49035', '29095', '13089', '06013', '01073', '06095', '29165', '39095', '24510', '12057', '26077', '12095', '36047', '01073', '28141', '06029', '18089', '29189', '48201', '04013', '12127', '48029', '13121', '12031', '18003', '08093', '20079', '32019', '17031', '44007', '53045', '39035', '51153', '05119', '54081', '45043', '26163', '21111', '48347', '06065', '37071', '35043', '12111', '47037', '51760', '42101', '06099', '47157', '45091', '29017', '39049', '35001', '29051', '55079', '20173', '06037', '28049', '55079', '13121', '24033', '25025', '36005', '20209', '47157', '24510', '17031', '22055', '17031', '48029', '22071', '39035', '42003', '37119', '55101', '12053', '26163', '34021', '36005', '30031', '13105', '13127', '06001', '12083', '27053', '17031', '42045', '41051', '48485', '42003', '51710', '51760', '26065', '22015', '12071', '06047', '17031', '22071', '12081', '21141', '13121', '13051', '29510', '06059', '34013', '48201', '01073', '17031', '12071', '12086', '08041', '48029', '48027', '17031', '34011', '12011', '12011', '06073', '01051', '53053', '30087', '06037', '42057', '21111', '12095', '17031', '48181', '12087', '12057', '06059', '06019', '06073', '31055', '12091', '29189', '36085', '34001', '06001', '17031', '13121', '06001', '08031', '09001', '51191', '01077', '37183', '22055', '37069', '12057', '37119', '12083', '39061', '17031', '22017', '39003', '29127', '48029', '17201', '12086', '32003', '17031', '17031', '12105', '12095', '35001', '17031', '12086', '12095', '51710', '21111', '12099', '06037', '08005', '11001', '30009', '12109', '47037', '12009', '17031', '12099', '22071', '47157', '26145', '06029', '01015', '35001', '04012', '37119', '47157', '22033', '42133', '39049', '06037', '55079', '37081', '45015', '09009', '18035', '45079', '36071', '41039', '39113', '47043', '34013', '29510', '24033', '17031', '21071', '39049', '26163', '42003', '01031', '12095', '06037', '39049', '45045', '48277', '55079', '48215', '42101', '42101', '42101', '12127', '06037', '26163', '17031', '17031', '41033', '01009', '48157', '11001', '11001', '24510', '13073', '17031', '39001', '37057', '29051', '53033', '32003', '48121', '01081', '36061', '20177', '55067', '04013', '26049', '17031', '17031', '51083', '39035', '34013', '06001', '08031', '29510', '12086', '13151', '32003', '26163', '37063', '47037', '47037', '29510', '24033', '48029', '22033', '55141', '28097', '06067', '19057', '06001', '29189', '01073', '17031', '22031', '29510', '11001', '31055', '47149', '55079', '06095', '12019', '36055', '12081', '27053', '12086', '32003', '22015', '17031', '26163', '01101', '37077', '24031', '15001', '37085', '27053', '39049', '12086', '39035', '48453', '17031', '51650', '39095', '01073', '13089', '17031', '17031', '01073', '17031', '05051', '11001', '51710', '37081', '48113', '37119', '26049', '04013', '06107', '13121', '54039', '06065', '31109', '47157', '13071', '12091', '18163', '17031', '22055', '45019', '36093', '51710', '17201', '48201', '18097', '29189', '01015', '24510', '42101', '13035', '48201', '42045', '17031', '17031', '36071', '26163', '06077', '47037', '12095', '39049', '17031', '22071', '47157', '22017', '55079', '34013', '17095', '17143', '17031', '17031', '36061', '34017', '48201', '42029', '42003', '55087', '36071', '47037', '25017', '12095', '17031', '36005', '17031', '51033', '37105', '39049', '36081', '53077', '32003', '48201', '34021', '18097', '42079', '06067', '24510', '37187', '42003', '17031', '18003', '18003', '32003', '06001', '34003', '55079', '12086', '26161', '36029', '08031', '21227', '01005', '47157', '12086', '48113', '12071', '45041', '37127', '02170', '04013', '06037', '36047', '27053', '06107', '48201', '04013', '05119', '26163', '17031', '47037', '39061', '12086', '25025', '11001', '39049', '12091', '05119', '22017', '42049', '28049', '18097', '17031', '35005', '27131', '06077', '06037', '12095', '09003', '17031', '42001', '04013', '17031', '36047', '17031', '06019', '54065', '13063', '06001', '10003', '42011', '17031', '26163', '51131', '26121', '39155', '41067', '17031', '17097', '06037', '36029', '17031', '21073', '13299', '51770', '34005', '12095', '17031', '24510', '12095', '18097', '12127', '47157', '42003', '09003', '06019', '51710', '12117', '34005', '21111', '53067', '17031', '13089', '42021', '34021', '28049', '24033', '09003', '20209', '24033', '48157', '48439', '51710', '42055', '39035', '21111', '40047', '17031', '17031', '06065', '21025', '13089', '17031', '45069', '45083', '37119', '17031', '36067', '48201', '31109', '48355', '06037', '41047', '17031', '26049', '48409', '16001', '44007', '24510', '17031', '12086', '06037', '13215', '26163', '32003', '29510', '05119', '17031', '01073', '36103', '06061', '28101', '04019', '17031', '44007', '12081', '39165', '18097', '29031', '13033', '27053', '22073', '40143', '47065', '37163', '36047', '04013', '06037', '47065', '29510', '48201', '32003', '39035', '04013', '17031', '42003', '17031', '29510', '06037', '17031', '25025', '13223', '36071', '06019', '24510', '37051', '48113', '22033', '22033', '47163', '29510', '06071', '28047', '17031', '13021', '17031', '12105', '54045', '48485', '12117', '42101', '26075', '26021', '20173', '11001', '17031', '48201', '24510', '12011', '51051', '39035', '39049', '13153', '53067', '29510', '34013', '48201', '48305', '06001', '39153', '53015', '17031', '42101', '11001', '06001', '17031', '39035', '06029', '01113', '48113', '09009', '06037', '06071', '27053', '47125', '11001', '37007', '06037', '19013', '26163', '06037', '12031', '48201', '22055', '22033', '36029', '20209', '34013', '37083', '48113', '05035', '29510', '51810', '06019', '34013', '12095', '17031', '34013', '18097', '06029', '17091', '17031', '39061', '12086', '36047', '22095', '45019', '18097', '22071', '48021', '34013', '36067', '39017', '42003', '12095', '09003', '17031', '06001', '17163', '13051', '12071', '47037', '18097', '17031', '05069', '42101', '17031', '04013', '12095', '25025', '12011', '18097', '47113', '01119', '12001', '17031', '36015', '24510', '01097', '56029', '13059', '26163', '08001', '29510', '28045', '47037', '17031', '13191', '53061', '29189', '44007', '55059', '25025', '32031', '48453', '17201', '12086', '18039', '34017', '17031', '17113', '18163', '34021', '12095', '12011', '26163', '18141', '06065', '09003', '13089', '22071', '12099', '26163', '37091', '36029', '47157', '22071', '22071', '13121', '06037', '51730', '06065', '13089', '26125', '17031', '06001', '42011', '29510', '04013', '34031', '11001', '17031', '26049', '08001', '28149', '29510', '06037', '13121', '06065', '26163', '18097', '36047', '47065', '51550', '29510', '39035', '06001', '12031', '34005', '17031', '48201', '35001', '21111', '17031', '53033', '06067', '39113', '37119', '51710', '42101', '12031', '48113', '55079', '48201', '29097', '31055', '36081', '37141', '06037', '51700', '21111', '06037', '34031', '18003', '51710', '31055', '27053', '36103', '31055', '17031', '13089', '13089', '06029', '11001', '06001', '45083', '09009', '42101', '48201', '29095', '48113', '51630', '26081', '06037', '53053', '17031', '34013', '39151', '01097', '09009', '38089', '42101', '37063', '25025', '13127', '09001', '51710', '06085', '55079', '48443', '39151', '34011', '17031', '04013', '54039', '01073', '12086', '17031', '06071', '17031', '48041', '35001', '36005', '17031', '06037', '17031', '48303', '17031', '13121', '29510', '37051', '01123', '09001', '06065', '34001', '37191', '55079', '42101', '13121', '39095', '17031', '37163', '27123', '42101', '13063', '06071', '32003', '06013', '34039', '36047', '10003', '17031', '17031', '37129', '09001', '29510', '24510', '12031', '24510', '48141', '02090', '19153', '51770', '17031', '36047', '17031', '24510', '20177', '42011', '37091', '06053', '24510', '37185', '37017', '29510', '48453', '42129', '05051', '42101', '26163', '29510', '39049', '51135', '12086', '48021', '12031', '01073', '37049', '26145', '29095', '22071', '34021', '06019', '18003', '34017', '26163', '34021', '18003', '22033', '51683', '48201', '56021', '34039', '48439', '29213', '06037', '42101', '37021', '11001', '22033', '26163', '18097', '12086', '37021', '12095', '12031', '42003', '42101', '51680', '34039', '17031', '28049', '17031', '17031', '01125', '26163', '02020', '21111', '47107', '48201', '47037', '53057', '34013', '06059', '22071', '28049', '24510', '01101', '06037', '17019', '37067', '48201', '48201', '22077', '39035', '01089', '17143', '24033', '06087', '45007', '13089', '12086', '06075', '48113', '28075', '06019', '48113', '17031', '34021', '17091', '37083', '47157', '06075', '48029', '22017', '06029', '27053', '26163', '22071', '08077', '28049', '06065', '17031', '01047', '29510', '06067', '36047', '24510', '26081', '06067', '17031', '39113', '37051', '26123', '17031', '17031', '17031', '25025', '17031', '17031', '26163', '48113', '17031', '22079', '06097', '01097', '06037', '17201', '04013', '17031', '06075', '01089', '54107', '21111', '29031', '12086', '13225', '42101', '06001', '40081', '17031', '51770', '39113', '45007', '12091', '22109', '17031', '13151', '26065', '12086', '28049', '39113', '01073', '04013', '06065', '06037', '26049', '08031', '25023', '28089', '20079', '39095', '36103', '24033', '17031', '40109', '36071', '42101', '36061', '21111', '22017', '32003', '22055', '47157', '05119', '28047', '34013', '17031', '42101', '17019', '36005', '29189', '06001', '17031', '12011', '48029', '45051', '01081', '01101', '42041', '42101', '22071', '16001', '34013', '17031', '47157', '53033', '48201', '42125', '24510', '48201', '16001', '08031', '09001', '28007', '29095', '06067', '06073', '48029', '12031', '12095', '48201', '17197', '12099', '42017', '47045', '12086', '13135', '17031', '06071', '06037', '42079', '01111', '05119', '12086', '06037', '17031', '12086', '12051', '06037', '42101', '28033', '45051', '36013', '36001', '21111', '02090', '17031', '06075', '22071', '17031', '01101', '12127', '12071', '17031', '23005', '42003', '08031', '22033', '22071', '21111', '06029', '29095', '36029', '06037', '06071', '48215', '06001', '17031', '17031', '12031', '17031', '06037', '08035', '29095', '42101', '24510', '06079', '08101', '13031', '39153', '04019', '06033', '12086', '54011', '34013', '12086', '06037', '26163', '17031', '18039', '17031', '17031', '24510', '12086', '35001', '21067', '02020', '29095', '12086', '06029', '47157', '13261', '51590', '48201', '06019', '06037', '06037', '36047', '37065', '12095', '24510', '22071', '32003', '13079', '06067', '13079', '42101', '22055', '34033', '04015', '18089', '37119', '24510', '05041', '17031', '17031', '26049', '37119', '28049', '17031', '53033', '17031', '42003', '36103', '47037', '48029', '47157', '41017', '48029', '45079', '45051', '37187', '06037', '01073', '17031', '36047', '28121', '20209', '49035', '02020', '37195', '37157', '41067', '26163', '36119', '17031', '01045', '17177', '04013', '22071', '22097', '29095', '17201', '13215', '12011', '22071', '48029', '13121', '17199', '39139', '01101', '48201', '09009', '48201', '40143', '13277', '29095', '29095', '34021', '12086', '22115', '28011', '48113', '45063', '46103', '29095', '17031', '20209', '37151', '05131', '17031', '17031', '24027', '17201', '12086', '17031', '41047', '05119', '48085', '06071', '12103', '06037', '06085', '04019', '06037', '17031', '12011', '04013', '27053', '39133', '26049', '06067', '17031', '26163', '48201', '21205', '01001', '08041', '10003', '17031', '28049', '34013', '48113', '06053', '12086', '29095', '06073', '21235', '42101', '27003', '13121', '22071', '28113', '17031', '08021', '48201', '13097', '48201', '34017', '40143', '12011', '51059', '12086', '16017', '06071', '17031', '49035', '45091', '37063', '32003', '47157', '53033', '28107', '06019', '47157', '22051', '17031', '01059', '17031', '17031', '47157', '47065', '12099', '17031', '13063', '17031', '17031', '13121', '22051', '37159', '48201', '48039', '17031', '48029', '17031', '48113', '13033', '21125', '18163', '18089', '11001', '17031', '28033', '12031', '01065', '13095', '29181', '24510', '51059', '29510', '53035', '22071', '47075', '48215', '45003', '17097', '45075', '37063', '40109', '17031', '39029', '55133', '04013', '34013', '24033', '22017', '27053', '35013', '47157', '11001', '37183', '08069', '27123', '37047', '17197', '12031', '01119', '21067', '24017', '39029', '26163', '12113', '24005', '12099', '12127', '28163', '22071', '42003', '39049', '40111', '05119', '48201', '40027', '45079', '39035', '24510', '13215', '01101', '48029', '01073', '28049', '48349', '06037', '13129', '22071', '02090', '19153', '24031', '12086', '12057', '12063', '06073', '45073', '26065', '47071', '17031', '39119', '36071', '37153', '12079', '29155', '45007', '06075', '28033', '06037', '17031', '06053', '17031', '51760', '24033', '49035', '36007', '13051', '12111', '06037', '27123', '39133', '13135', '42101', '18141', '17031', '06037', '06065', '36067', '51710', '16041', '19169', '26163', '55025', '39035', '06037', '42101', '28075', '39035', '12086', '20091', '36055', '26163', '40143', '24510', '22071', '48201', '11001', '24510', '20079', '06029', '37119', '12057', '12031', '17031', '39155', '12117', '17031', '21067', '17165', '06095', '18089', '37081', '51700', '48201', '01097', '39035', '06107', '48467', '39061', '12095', '48201', '48439', '32031', '06065', '42101', '48165', '20177', '40119', '06067', '06001', '45041', '06025', '44007', '13215', '36029', '12086', '17031', '48449', '27053', '55079', '12086', '12086', '17089', '17089', '22071', '17031', '05051', '47157', '39049', '42003', '36067', '40113', '27007', '48245', '17031', '22097', '31055', '06073', '17031', '32003', '24005', '47157', '22051', '48453', '48427', '26163', '36005', '36047', '13121', '22071', '39095', '06095', '26163', '17031', '21111', '29095', '22033', '48439', '55073', '13089', '22001', '13089', '12031', '24017', '06037', '29510', '06103', '06019', '48201', '20173', '42043', '48215', '39035', '26163', '42077', '18089', '47093', '39035', '22071', '48231', '29019', '39061', '17031', '34031', '48201', '24033', '09001', '51083', '12086', '18003', '40145', '55079', '47157', '37083', '06037', '13127', '12117', '31055', '13121', '12099', '28049', '06075', '37165', '51760', '09003', '39151', '17031', '45055', '45019', '48473', '12057', '40109', '40111', '48479', '06013', '55079', '06037', '06067', '17031', '19153', '48303', '42101', '06037', '12031', '26163', '17031', '47157', '22019', '19153', '06001', '48303', '27123', '17031', '48201', '37167', '37035', '13127', '12086', '48303', '18097', '24510', '39081', '28033', '12005', '39035', '51760', '12099', '18003', '12095', '26145', '48029', '06071', '17031', '47141', '41051', '21111', '48201', '06095', '12011', '09001', '13121', '17031', '20125', '28133', '29510', '13089', '06065', '04019', '01047', '18097', '42101', '12057', '17195', '08013', '29095', '17031', '17031', '06065', '01073', '06071', '17031', '42101', '06095', '51710', '37139', '17201', '11001', '39049', '12009', '10001', '48201', '39049', '42101', '06019', '22071', '13121', '37063', '47037', '53033', '12071', '21111', '45053', '29510', '12009', '22071', '06031', '06037', '34031', '34011', '06085', '12086', '12086', '12019', '42133', '04019', '48201', '48113', '36067', '13051', '45041', '06001', '05119', '13157', '48201', '12086', '29189', '34013', '25025', '42045', '13245', '09003', '06037', '17031', '51810', '24031', '06037', '06067', '06037', '47147', '26163', '20177', '39049', '22055', '06073', '12031', '45051', '17031', '48113', '19155', '06037', '12031', '32003', '37047', '26163', '06085', '26049', '13127', '06067', '54011', '12037', '45079', '28049', '17031', '18097', '06019', '48201', '17031', '53077', '17031', '06019', '44007', '06095', '20209', '48201', '11001', '34021', '42101', '22009', '05149', '29095', '05069', '39089', '21113', '51510', '36055', '17031', '05031', '12055', '22073', '48439', '13175', '05119', '19163', '51760', '34021', '22071', '36005', '45083', '12055', '17031', '45029', '47037', '39013', '34021', '01015', '04013', '37155', '35013', '17031', '47037', '04013', '17031', '12086', '53063', '42101', '45083', '04013', '40031', '17031', '17031', '19153', '36029', '17043', '47157', '40145', '27123', '26163', '42101', '06037', '47157', '06015', '29510', '34021', '37183', '47047', '06029', '12086', '10003', '17031', '05131', '45063', '12095', '11001', '17197', '12031', '36061', '37049', '28085', '01113', '48113', '12086', '26145', '28059', '17167', '53033', '51143', '08031', '42101', '34031', '05093', '13089', '17031', '17031', '17031', '36055', '21111', '04013', '48231', '39017', '42003', '22071', '51650', '27003', '01089', '42101', '28143', '28065', '29510', '18097', '17031', '51041', '48479', '06037', '06037', '26021', '06111', '29510', '22071', '22071', '29095', '25013', '17031', '17031', '18175', '29510', '26163', '18003', '12095', '12009', '12057', '06019', '06071', '48409', '42071', '49035', '48029', '24005', '29095', '42131', '22071', '21117', '36047', '34013', '11001', '48439', '47157', '12111', '48027', '47065', '36029', '17031', '48201', '12011', '24510', '17031', '21121', '06037', '26163', '17031', '29189', '06067', '32003', '24510', '24510', '13151', '45023', '06075', '39151', '51510', '01101', '35039', '13139', '26163', '22071', '47001', '17031', '51760', '39151', '21111', '17031', '18177', '06067', '28161', '11001', '40143', '36047', '17031', '17031', '17031', '39049', '32031', '11001', '29510', '17031', '45051', '22017', '51760', '13127', '17031', '48201', '16027', '47031', '42049', '27123', '22079', '17031', '01097', '17031', '17031', '39113', '47093', '12011', '54025', '47093', '13121', '13067', '21111', '17089', '37147', '36061', '12011', '06073', '32003', '34005', '27123', '37081', '36061', '32003', '26163', '22017', '13057', '22017', '18097', '29510', '39151', '11001', '47065', '45031', '17031', '17031', '37147', '13095', '12086', '01101', '22017', '06001', '39153', '24510', '45057', '32003', '47157', '48029', '47157', '01073', '24510', '17031', '36005', '12071', '34013', '17091', '37067', '34013', '05119', '06001', '47037', '34017', '29095', '17031', '34013', '17031', '13121', '45047', '25025', '01073', '22071', '26125', '37051', '01001', '26163', '01073', '05035', '34017', '25025', '12105', '22033', '06083', '51650', '12095', '39071', '37183', '42045', '45029', '18089', '36005', '12011', '04013', '36081', '29510', '13051', '23025', '17031', '17031', '28075', '48463', '29510', '48113', '48113', '42017', '13215', '34013', '04013', '39061', '42101', '08031', '17031', '19113', '17119', '45021', '17031', '34013', '37119', '36005', '36045', '51025', '36055', '17031', '17031', '32003', '51153', '29510', '06065', '01073', '34021', '42003', '39035', '29510', '45077', '17031', '48029', '17031', '42101', '06001', '21017', '29095', '29019', '18097', '12099', '55079', '34021', '04013', '06037', '38105', '12099', '06037', '37143', '12055', '17031', '39049', '04012', '48201', '17031', '51760', '29510', '12011', '29021', '18097', '35001', '06067', '17031', '01101', '17031', '36005', '39061', '22033', '47157', '17031', '42049', '05119', '37183', '01073', '37057', '37081', '12031', '36085', '37081', '51510', '18097', '12086', '17031', '17031', '17031', '48365', '17031', '06029', '39153', '10003', '13301', '36029', '12086', '37081', '42101', '37159', '39061', '15001', '51810', '29225', '13157', '48141', '17115', '08041', '24005', '06047', '17031', '32003', '06037', '06019', '51710', '53077', '06115', '13179', '37183', '24510', '01073', '18141', '20177', '11001', '39155', '13051', '06075', '12091', '42045', '30003', '34021', '12073', '17031', '45005', '42045', '01073', '34029', '12081', '12086', '48423', '05119', '06001', '28151', '36005', '36005', '17163', '05123', '20209', '11001', '12086', '45083', '29510', '29510', '17031', '40031', '34011', '06037', '06067', '48167', '18089', '29189', '27053', '42101', '48201', '24510', '42045', '06085', '06067', '17031', '36021', '42077', '51800', '55059', '13001', '20045', '06037', '05119', '48029', '18097', '29510', '42075', '36103', '05107', '17031', '12031', '42051', '34023', '48029', '17031', '17031', '06037', '06029', '39035', '29510', '12086', '17031', '22017', '17031', '36047', '17031', '34013', '24510', '48113', '34013', '48439', '55079', '47157', '37083', '22061', '17031', '06071', '27123', '31109', '17143', '29189', '21111', '48113', '36047', '36119', '22071', '47037', '17201', '39035', '18177', '05069', '25009', '29510', '13075', '12083', '55079', '51087', '06067', '24510', '18163', '17197', '42101', '35009', '13089', '17031', '06029', '48201', '06107', '06067', '17031', '17031', '37081', '17031', '10001', '42003', '48439', '48113', '29510', '34013', '11001', '17031', '21111', '47077', '17031', '39049', '06037', '05037', '17031', '06095', '19153', '55079', '01015', '26049', '48201', '12099', '48201', '26121', '06045', '22071', '17031', '39035', '04013', '12071', '39035', '06067', '17197', '06059', '42101', '20209', '17031', '01047', '48113', '17031', '29189', '48085', '25025', '51760', '22071', '39035', '28113', '02020', '45075', '47157', '12071', '42101', '11001', '42133', '53063', '47125', '42101', '45079', '51650', '15003', '42003', '06065', '36029', '21029', '37063', '28049', '51730', '26065', '17031', '01083', '48029', '13021', '26163', '26081', '40051', '22051', '40047', '42045', '17097', '48037', '48201', '34013', '13215', '34023', '17031', '12073', '01089', '20209', '18089', '39061', '26163', '21111', '37063', '21111', '17031', '12086', '17043', '13031', '45029', '17183', '18089', '06029', '17031', '09003', '09009', '09003', '49033', '24510', '26163', '13103', '39095', '47037', '12073', '36067', '47157', '01073', '42101', '45075', '46083', '24510', '24510', '42101', '05045', '39085', '22071', '47109', '06077', '11001', '06037', '47157', '26125', '17031', '37045', '13059', '13215', '48029', '20045', '01073', '22055', '53033', '12115', '06019', '12086', '17031', '48041', '42101', '42101', '06001', '12031', '04021', '06037', '05091', '06037', '24510', '01025', '17143', '13021', '12063', '17031', '28049', '51760', '48439', '04013', '26163', '18097', '17031', '29019', '47157', '06001', '54039', '47157', '47065', '06037', '06075', '48381', '06085', '39095', '29095', '17031', '42101', '47157', '11001', '39087', '06071', '06059', '28043', '47037', '21231', '28095', '04013', '06037', '48201', '28075', '51650', '36047', '24025', '05123', '01089', '26075', '08041', '29179', '08031', '36005', '08069', '47157', '36005', '42045', '06077', '12057', '48201', '48201', '47037', '12031', '45007', '48201', '11001', '51510', '09003', '24045', '01017', '48201', '22105', '40143', '25009', '47093', '48113', '39049', '13089', '21111', '42101', '20087', '17031', '37047', '06075', '55005', '21111', '54089', '25025', '48201', '39095', '51730', '01101', '08041', '39049', '08041', '22071', '48453', '37081', '48439', '36005', '31185', '06001', '18089', '45031', '08001', '01015', '18089', '29510', '53033', '17031', '27053', '36029', '11001', '06037', '29189', '22103', '06029', '39099', '48453', '51710', '51710', '06037', '48493', '21117', '26163', '45029', '06019', '51710', '24510', '20209', '06065', '49035', '41019', '42081', '29131', '13063', '06037', '04019', '18003', '47163', '48113', '49049', '42101', '39061', '37159', '26077', '12105', '06029', '39113', '04013', '29095', '18097', '48113', '22071', '32003', '37051', '47157', '29510', '18089', '47157', '42101', '06019', '37107', '13121', '22017', '47157', '40109', '48423', '28075', '06103', '06013', '17031', '55079', '39049', '54039', '17031', '13215', '22045', '39049', '53033', '27053', '37191', '47065', '42101', '32003', '26151', '18097', '47157', '08101', '34021', '01073', '29225', '42101', '42101', '37051', '08031', '51041', '42033', '39035', '28085', '17031', '12009', '29510', '47159', '30089', '51177', '17031', '04019', '48201', '04013', '22033', '06031', '39095', '26163', '37085', '37081', '39095', '28125', '39035', '29189', '48201', '04013', '26163', '42003', '06037', '26163', '17031', '47037', '06029', '06037', '34001', '34017', '06019', '29510', '12095', '47157', '24510', '42003', '05051', '25025', '17031', '48201', '48201', '35045', '17019', '48201', '06047', '48439', '37119', '25025', '20173', '06077', '39049', '48201', '28153', '13121', '12033', '28035', '48113', '21111', '21111', '39049', '24510', '09003', '24510', '29189', '06037', '37183', '28007', '19153', '26125', '12095', '17031', '12009', '45075', '53035', '48027', '39035', '28001', '13215', '06037', '22055', '42101', '13121', '32003', '26163', '22001', '29510', '51059', '06019', '04013', '22097', '06095', '01073', '28149', '36063', '01109', '04013', '45021', '18057', '22017', '12086', '35001', '47157', '31055', '42003', '32003', '32003', '48201', '25025', '13215', '06037', '48201', '12115', '29510', '32003', '12105', '08035', '12095', '17031', '06113', '34025', '36029', '05145', '47045', '06019', '01089', '39049', '06037', '48355', '48113', '05095', '17031', '48439', '20065', '29095', '28035', '06037', '39043', '01101', '47157', '36005', '26067', '45079', '01011', '21067', '22065', '06089', '12011', '12086', '48167', '12031', '45007', '51810', '04019', '12083', '17031', '36047', '37097', '06065', '36005', '04019', '37119', '01113', '29169', '36111', '39081', '40109', '06099', '11001', '37119', '21111', '47037', '27053', '41039', '17031', '18003', '01089', '01063', '02020', '37151', '48201', '22073', '53077', '06075', '12001', '48113', '17031', '45091', '11001', '55079', '06037', '40147', '21111', '48041', '01015', '42043', '06019', '28047', '32003', '39153', '55073', '17031', '36061', '12095', '21111', '06071', '48113', '45007', '39097', '06013', '49049', '34021', '12053', '01103', '42101', '21157', '13215', '06107', '26023', '13093', '22033', '11001', '04013', '12083', '12083', '48201', '51650', '39049', '24001', '06037', '21227', '29510', '08031', '47093', '42051', '42011', '37083', '18097', '40023', '36081', '05123', '13089', '24510', '48201', '17031', '29510', '29510', '48201', '21111', '06047', '42101', '40027', '26163', '06073', '29510', '55019', '55025', '39035', '45033', '08031', '26049', '13021', '17031', '28059', '37155', '04013', '12099', '08041', '32003', '18003', '01101', '12099', '37183', '09001', '01077', '29023', '06019', '13151', '21115', '45079', '12071', '26163', '11001', '42101', '18163', '48113', '13089', '22071', '49035', '12011', '42101', '17097', '37129', '37129', '26125', '20209', '01097', '17031', '47157', '21235', '48029', '48201', '24510', '21003', '47065', '26017', '04013', '01101', '17031', '29189', '39153', '12099', '39085', '26163', '42101', '21117', '39061', '06037', '11001', '04013', '06081', '42101', '26163', '27123', '09001', '42077', '28121', '48201', '42101', '13215', '29510', '48113', '12099', '36047', '17201', '12086', '17201', '17031', '16017', '29095', '48459', '29189', '17031', '29083', '17031', '01113', '01073', '42101', '06029', '11001', '06055', '37007', '06037', '18141', '40133', '12083', '26049', '26145', '13021', '17019', '12057', '04013', '24510', '06099', '12003', '53053', '01073', '12083', '36047', '37181', '39151', '55079', '48201', '28001', '17031', '21111', '42101', '32003', '45079', '17021', '24037', '06001', '06075', '28081', '42049', '37097', '12127', '48029', '49035', '12099', '51650', '48201', '12095', '17031', '45029', '45013', '29189', '48085', '06065', '34007', '28029', '32003', '34025', '12095']

In [ ]:
#check county codes 
df_fips['fips'] = county_fips

#aggregate joint counties
dict_fips = {}
for f, n in zip(df_fips['fips'], df_fips['n_casualties']):
    if f in dict_fips:
        dict_fips[f] += n
    else:
        dict_fips[f] = n

In [ ]:
#split data by county 
#aggregate density by state
#choropleth map split by county 
colorscale = ["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]

endpts = list(np.linspace(1, 12, len(colorscale) - 1))
fips = list(dict_fips.keys())
values = list(dict_fips.values())

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 1},
    asp=2.9, title='USA Casualties %',
    legend_title='Casualties %'
)
iplot(fig, filename='choropleth_full_usa')


In [ ]:
#plot density map on U.S. for number of casualities by long/lat
'''
limits = [(0,2),(3,10),(11,20),(21,50),(50,200)]
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
density_cas = []
scale = 1

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[(df['n_casualties'] >= lim[0]) & (df['n_casualties'] <= lim[1])]
    cas = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['longitude'],
        lat = df_sub['latitude'],
        text = "Number of guns involved: "+str(df_sub['n_guns_involved']),
        marker = dict(
            size = df_sub['n_casualties']/scale,
            # sizeref = 2. * max(df_sub['pop']/scale) / (25 ** 2),
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    density_cas.append(cas)

layout = dict(
        title = '2013-2018 Gun Violence<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict(data=density_cas, layout=layout)
iplot(fig, show_link=False)
'''

## Ethics and Privacy

## Dicussion

## Moving Forward